<font color=blue size=6 style='bold'> Indeed Job Data Scrapping Into CSV </font>

## Configure Search

In [1]:

ADVANCE_SEARCH = True  # For Advance Search
if ADVANCE_SEARCH:
    adv_url = "https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&vjk=0c280f5c7a18a0b8"
else:
    what_job = 'Python Django'    # For Basic Search
    where = 'ahmedabad, gujarat' 

    
Filename = 'Indeed_Job_Data2.csv'
save_data = True



In [2]:
if ADVANCE_SEARCH:
    

    i = adv_url.find("vjk=")
    adv_url = adv_url[:i-1]
    if "&start" in adv_url:
        adv_url=adv_url[:-2]+"{}"
    else:
        adv_url+="&start={}"


    def get_adv_url(adv_url, page):
        url = adv_url.format((page-1)*10)
        return url


In [3]:
import csv 
from datetime import datetime
import requests
from bs4 import BeautifulSoup as bs
import time 
from datetime import date
today = date.today()


## Formats URL For Each Page
####  func get_url
#### input =  position/post of job , location of company, page number to access
#### returns url - for the required job, location and page number

In [4]:
def get_url(position, location, page):
    "Generate Url From Position and Location Arguments"
    position=position.replace(" ","%20")
    position=position.replace(",","%2C")
    location=location.replace(" ","%20")
    location=location.replace(",","%2C")
    location,position = location.lower(),position.lower()
    template = "https://in.indeed.com/jobs?q={}&l={}&start={}"
    url = template.format(position, location, (page-1)*10)
    return url        

## Returns details of required card
####  func get_record 
#### Input = card 
#### return record = (job_title,company_name,company_location,salary,summary,post_date,today,job_url)

In [5]:
def get_record(card):
    
    atag = card.h2.a
    atag_span = card.h2.a.span # Hierarchy of tags H2->a-> span-class='title'
    job_title = atag_span.get('title') # extracted from class='title' || atag.text can also work #TITLE
    job_url = "https://www.indeed.com" + atag.get("href") #Job_URL
    company_name = card.find("span",'companyName').text.strip() #Company Name
    company_location = card.find("div",'companyLocation').text.strip()
    today = datetime.today().strftime("%y-%m-%d")
    summary = card.find("div",'job-snippet').text.strip().replace("\n",' ') #Summary
    post_date = card.find("span",'date').text.replace("Employer","Employer ").replace("Posted","Posted ") # Posted before  
    try:
        salary = card.find("div",'attribute_snippet').text #Salary_Range
    except AttributeError:
        salary = 'NA'
    record = {'job_title':job_title,'company_name':company_name,"company_location":company_location,'salary':salary,'summary':summary,'post_date':post_date,'today':today,'job_url':job_url}
    return record

In [6]:
records = []
def save_records(cards):
    for card in cards:
        record = get_record(card)
        records.append(record)
    

## Itterates through each page and Checks if Page Exists

In [10]:
records = []
begin = time.time()  # Starts timer from here !!!
i=0
while True:
    i+=1  
    try:
        if ADVANCE_SEARCH:
            url = get_adv_url(adv_url, i)
            print(url)
        else:
            url = get_url(what_job, where,i)
        response = requests.get(url)
        soup = bs(response.text, 'html.parser')
        # Check if its a new page -> to check if page exist
        page_number = int(soup.select('b[aria-current]')[0].text)
        if True:         # To print Page number, Itteration number and time elapsed
            now = time.time()
            print('Page Number, Itteration :', page_number,i,' Time Taken Till Now:',int(now-begin),'s')
        if page_number != i:
            print("Last Page Reached. Last Page is ",i-1 )
            break
        # If page exists get job cards
        cards = soup.find_all('div','job_seen_beacon')
        # calling the function to Save the Details of the job cards
        save_records(cards)
        if False:     # To show number of records fetched till this page
            print('Records Fetched Till Now:',len(records))
        
    except:
        break
time.sleep(1)
now = time.time()
print("Total Jobs Obtained : ",len(records))
print("Total Page Loaded :",i-1)
print("Total Time Taken :", int(now-begin),'s')

https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=0
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=10
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=20
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=30
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=40
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=50
https://in.indeed.com/jobs?q=python%20django%20%E2%82%B93%2C50%2C000&l=ahmedabad%2C%20gujarat&start=60
Last Page Reached. Last Page is  6
Total Jobs Obtained :  89
Total Page Loaded : 6
Total Time Taken : 33 s


## Save the Fetched Data into CSV

In [8]:
fields = list(records[0].keys()) 

In [9]:
if save_data:
    import csv # Don't Forget To Change The File Name
    with open(Filename, 'w') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames = fields)
        writer.writeheader()
        writer.writerows(records)